In [51]:
import pandas as pd
from functools import reduce

In [2]:
ROOT_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim/"
DF_NAMES = (
    "CH4_df",
    "CO2_df",
    "HFC_df",
    "NF3_df",
    "PFC_df",
    "SF6_df",
    "N20_df",
)

CH4_df = pd.read_pickle(ROOT_DIR + DF_NAMES[0] + ".pkl")
CO2_df = pd.read_pickle(ROOT_DIR + DF_NAMES[1] + ".pkl")
HFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[2] + ".pkl")
NF3_df = pd.read_pickle(ROOT_DIR + DF_NAMES[3] + ".pkl")
PFC_df = pd.read_pickle(ROOT_DIR + DF_NAMES[4] + ".pkl")
SF6_df = pd.read_pickle(ROOT_DIR + DF_NAMES[5] + ".pkl")
N20_df = pd.read_pickle(ROOT_DIR + DF_NAMES[6] + ".pkl")

In [12]:
list_of_dfs = [CH4_df,
CO2_df,
HFC_df,
NF3_df,
PFC_df,
SF6_df,
N20_df]

In [39]:
DF_NAMES[0][:3]

'CH4'

In [32]:
test_df = pd.merge(N20_df, CO2_df, on = "Country or Area")

In [34]:
CO2_df.head()

,Country or Area,Year,Value
0,Australia,2018,415953.95
1,Australia,2017,415097.43
2,Australia,2016,411031.53
3,Australia,2015,401554.76
4,Australia,2014,394116.89


In [36]:
CO2_df.rename({"Country or Area":"Country", "Year":"Year", "Value":"CO2 Emissions"}, axis = 1, inplace = True)

In [40]:
for df, name in zip(list_of_dfs, DF_NAMES):
    df.rename({"Country or Area":"Country", "Year":"Year", "Value":f"{name[:3]} Emissions"}, axis = 1, inplace = True)

In [54]:
CO2_df.head()

,Country,Year,CO2 Emissions
0,Australia,2018,415953.95
1,Australia,2017,415097.43
2,Australia,2016,411031.53
3,Australia,2015,401554.76
4,Australia,2014,394116.89
